In [4]:
# necessary to start the model
!ollama run llama2
# may take a moment the first time running

⠙ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠴ ⠧ ⠇ ⠇ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ >>> Send a message (/? for help)
Use Ctrl + d or /bye to exit.
>>> Send a message (/? for help)Send a message (/? for help)

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")
llm("The first man on the moon was ...")

/Users/I548692/.pyenv/versions/3.10.5/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nThe first man to walk on the moon was Neil Armstrong. He stepped foot on the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" as he became the first person to set foot on the lunar surface.'

## Prepare Documents
Only needs to be run once, then will persist in memory

In [8]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = DirectoryLoader("texts", glob="*.txt", show_progress=True)
docs = loader.load()
len(docs)

In [6]:
raw_text = ''
for i, doc in enumerate(docs):
    text = doc.page_content
    if text:
        raw_text += text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_text(raw_text)
len(texts)

In [11]:
texts[0]

'A Appendix: Instructions This appendix provides the English translations of the instructions for the 4I treatment.'

### Prepare Database

In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs,

)

/Users/I548692/.pyenv/versions/3.10.5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
%%time
### Make the chroma and persiste to disk
db = Chroma.from_texts(texts,
                       embedding_function,
                       persist_directory="./chroma_db")



### load from disk
# db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

In [ ]:
query = "Tell me about Universal Studios Singapore?"

db.similarity_search(query, k=5)